In [2]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Wed Jun 26 10:29:17 PDT 2019
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
roadrunner

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                16
On-line CPU(s) list:   0-15
Thread(s) per core:    2
Core(s) per socket:    4
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 26
Model name:            Intel(R) Xeon(R) CPU           E5520  @ 2.27GHz
Stepping:              5
CPU MHz:               2394.000
CPU max MHz:           2394.0000
CPU min MHz:           1596.0000
BogoMIPS:              4521.78
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              8192K
NUMA node0 CPU(s):     0-15
Flags:                 fpu vme de pse tsc msr pae m

No LSB modules are available.


In [3]:
# Set working directories - %env useful for bash
data_dir = "/home/sam/data/P_generosa/P_generosa_genomes"
%env data_dir = /home/sam/data/P_generosa/P_generosa_genomes
work_dir = "/home/sam/analyses/20190626_Pgenerosa_v074_repeatmodeler"
%env work_dir = /home/sam/analyses/20190626_Pgenerosa_v074_repeatmodeler

env: data_dir=/home/sam/data/P_generosa/P_generosa_genomes
env: work_dir=/home/sam/analyses/20190626_Pgenerosa_v074_repeatmodeler


In [4]:
# Set file paths/names
%env Pgenerosa_v074_fasta = /home/sam/data/P_generosa/P_generosa_genomes/Pgenerosa_v074.fa
%env rptm_db_name = Pgenerosa_v074

env: Pgenerosa_v074_fasta=/home/sam/data/P_generosa/P_generosa_genomes/Pgenerosa_v074.fa
env: rptm_db_name=Pgenerosa_v074


In [5]:
# Set program paths
%env rptm = /home/shared/RepeatModeler-open-1.0.11

env: rptm=/home/shared/RepeatModeler-open-1.0.11


In [6]:
%%bash
mkdir --parents "${work_dir}"

In [7]:
%%bash
cd "${data_dir}"
time \
rsync \
--archive \
owl:/volume1/web/halfshell/genomic-databank/Pgenerosa_v074.fa \
.
ls -lh

# Uncomment following line(s) to download from web
# wget https://owl.fish.washington.edu/halfshell/genomic-databank/Pgenerosa_v074.fasta

total 5.5G
-rw-r--r-- 1 sam users 2.1G Feb 11 12:13 Pgenerosa_v070.fa
-rw-rw-rw- 1 sam users  20M Feb 11 13:49 Pgenerosa_v070.fa.fai
-rw-rw-r-- 1 sam sam   1.4G Feb 11 13:51 Pgenerosa_v071.fasta
-rw-rw-r-- 1 sam sam   905K Feb 11 16:38 Pgenerosa_v071.fasta.fai
-rw-rw-r-- 1 sam sam   1.2G Feb 11 13:52 Pgenerosa_v073.fasta
-rw-rw-r-- 1 sam sam    87K Feb 11 16:39 Pgenerosa_v073.fasta.fai
-rw-rw-rw- 1 sam users 914M Jun 26 08:49 Pgenerosa_v074.fa



real	2m25.346s
user	0m15.348s
sys	0m6.052s


#### Create RepeatModeler databases

In [8]:
%%bash
cd "${work_dir}"

time \
perl ${rptm}/BuildDatabase \
-name "${rptm_db_name}" \
-engine ncbi \
"${Pgenerosa_v074_fasta}" \
>& database_build_run.out

echo "------------------------------------------------------------------------"
echo ""
echo "------------------------------------------------------------------------"
cat ${work_dir}/database_build_run.out

------------------------------------------------------------------------

------------------------------------------------------------------------
Can't open perl script "/home/shared/RepeatModeler-open-1.0.11/BuildDatabase": No such file or directory



real	0m0.200s
user	0m0.000s
sys	0m0.004s


#### Run RepeatModeler

In [7]:
%%bash

cd ${work_dir}
time \
perl ${rptm}/RepeatModeler \
-database "${work_dir}"/"${rptm_db_name}" \
-engine ncbi \
-pa 16 \
>& run.out

sed '/^Subject:/ s/ / repeatmodeler JOB COMPLETE/' ~/.default-subject.mail | msmtp "$EMAIL"


real	2056m18.020s
user	20838m22.216s
sys	407m24.308s


In [8]:
%%bash
tail -n 50 ${work_dir}/run.out

  - Saving elements to a file...
    - 16 elements found.
Element Gathering: 00:00:00 (hh:mm:ss) Elapsed Time
Refining family-1617 model...
  - numRounds = 7
  - Consensus Length = 233 ( orig = 233 )
  - Avg Kimura Divergence = 0.01
  - Unaligned sequences = 0 ( orig = 0 )
  Build Consensus: 0:0:1 Elapsed Time
Refinement: 00:00:02 (hh:mm:ss) Elapsed Time

Processing RECON family: 6568
  - Saving elements to a file...
    - 16 elements found.
Element Gathering: 00:00:00 (hh:mm:ss) Elapsed Time
Refining family-6568 model...
  - numRounds = 6
  - Consensus Length = 404 ( orig = 417 )
  - Avg Kimura Divergence = 0.01
  - Unaligned sequences = 3 ( orig = 3 )
  Build Consensus: 0:0:2 Elapsed Time
Refinement: 00:00:03 (hh:mm:ss) Elapsed Time
Family Refinement: 00:23:28 (hh:mm:ss) Elapsed Time
Round Time: 16:14:42 (hh:mm:ss) Elapsed Time

Discovery complete: 2001 families found
Classifying Repeats...
RepeatClassifier Version open-1.0.11
Search Engine = ncbi
  - Looking for Simple and Low Compl

In [9]:
%%bash
cd /home/sam/analyses/
rsync --archive --relative ./"${work_dir}" gannet:/volume1/web/Atumefaciens

sed '/^Subject:/ s/ / rsync JOB COMPLETE/' ~/.default-subject.mail | msmtp "$EMAIL"